In [9]:
import paho.mqtt.client as mqtt
import json
import requests

In [10]:
# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, reason_code, properties):
    print(f"Connected with result code {reason_code}")
    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe("$SYS/#")
    client.subscribe("frigate/events")

# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    if (msg.topic == 'frigate/events'):
        print(f'{msg.topic} {msg.payload}')
        # payload = json.loads(str(msg.payload)[2:-1])
        # print(f'{payload["after"]["id"]} {payload["after"]["end_time"]}')
    else:
        print(msg.topic+" "+str(msg.payload))

mqttc = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)
mqttc.on_connect = on_connect
mqttc.on_message = on_message

mqttc.connect("localhost", 1883, 60)

# Blocking call that processes network traffic, dispatches callbacks and
# handles reconnecting.
# Other loop*() functions are available that give a threaded interface and a
# manual interface.
mqttc.loop_forever()

Connected with result code Success


KeyboardInterrupt: 

In [ ]:
string = """{"before": {"id": "1710925666.831342-hep4y7", "camera": "test", "frame_time": 1710925677.235791, "snapshot": {"frame_time": 1710925667.026031, "box": [441, 166, 529, 357], "area": 16808, "region": [0, 0, 864, 864], "score": 0.7551310658454895, "attributes": []}, "label": "\\u0447\\u0435\\u043b\\u043e\\u0432\\u0435\\u043a", "sub_label": null, "top_score": 0.7451910972595215, "false_positive": false, "start_time": 1710925666.831342, "end_time": null, "score": 0.6662592887878418, "box": [436, 176, 519, 370], "area": 16102, "ratio": 0.42783505154639173, "region": [0, 0, 640, 640], "stationary": true, "motionless_count": 51, "position_changes": 1, "current_zones": [], "entered_zones": [], "has_clip": true, "has_snapshot": false, "attributes": {}, "current_attributes": []}, "after": {"id": "1710925666.831342-hep4y7", "camera": "test", "frame_time": 1710925686.628307, "snapshot": {"frame_time": 1710925667.026031, "box": [441, 166, 529, 357], "area": 16808, "region": [0, 0, 864, 864], "score": 0.7551310658454895, "attributes": []}, "label": "\\u0447\\u0435\\u043b\\u043e\\u0432\\u0435\\u043a", "sub_label": null, "top_score": 0.7451910972595215, "false_positive": false, "start_time": 1710925666.831342, "end_time": 1710925691.831247, "score": 0.7020306587219238, "box": [436, 180, 515, 355], "area": 13825, "ratio": 0.4514285714285714, "region": [0, 0, 680, 680], "stationary": true, "motionless_count": 98, "position_changes": 1, "current_zones": [], "entered_zones": [], "has_clip": true, "has_snapshot": false, "attributes": {}, "current_attributes": []}, "type": "end"}"""
json.loads(string)

{'before': {'id': '1710925666.831342-hep4y7',
  'camera': 'test',
  'frame_time': 1710925677.235791,
  'snapshot': {'frame_time': 1710925667.026031,
   'box': [441, 166, 529, 357],
   'area': 16808,
   'region': [0, 0, 864, 864],
   'score': 0.7551310658454895,
   'attributes': []},
  'label': 'человек',
  'sub_label': None,
  'top_score': 0.7451910972595215,
  'false_positive': False,
  'start_time': 1710925666.831342,
  'end_time': None,
  'score': 0.6662592887878418,
  'box': [436, 176, 519, 370],
  'area': 16102,
  'ratio': 0.42783505154639173,
  'region': [0, 0, 640, 640],
  'stationary': True,
  'motionless_count': 51,
  'position_changes': 1,
  'current_zones': [],
  'entered_zones': [],
  'has_clip': True,
  'has_snapshot': False,
  'attributes': {},
  'current_attributes': []},
 'after': {'id': '1710925666.831342-hep4y7',
  'camera': 'test',
  'frame_time': 1710925686.628307,
  'snapshot': {'frame_time': 1710925667.026031,
   'box': [441, 166, 529, 357],
   'area': 16808,
   '

In [ ]:
"""
1710880842.594859-kakz56
"""

'\n1710880842.594859-kakz56\n'

In [ ]:
id = '1710880842.594859-kakz56'

In [ ]:
response = requests.get(f'http://localhost:5000/api/events/{id}/clip.mp4')

with open(f'{id}.mp4', 'wb') as video:
  video.write(response.content)

In [ ]:
# Тута определение скорости

max_estimated_speed = 40
if max_estimated_speed > 30:
    url = f'http://localhost:5000/api/events/{id}/sub_label'
    data = {
        "subLabel": f"Макс. скорость: {max_estimated_speed} км/ч"
    }

    response = requests.post(url=url, json=data)
    response.json()

{'message': 'Event 1710880842.594859-kakz56 sub label set to Макс. скорость: 40 км/ч',
 'success': True}

### Заготовка

In [1]:
import paho.mqtt.client as mqtt
import json
import requests

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
CAMERAS = os.getenv('CAMERAS').split(' ')
ZONES = os.getenv('ZONES').split()
LABELS = os.getenv('LABELS').split()
DURATION = int(os.getenv('DURATION'))
MAX_SPEED = int(os.getenv('MAX_SPEED'))

In [4]:
LABELS

['no_vest', 'no_hemlet']

In [5]:
def speed_estimation(id):
    response = requests.get(f'http://localhost:5000/api/events/{id}/clip.mp4')

    with open(f'{id}.mp4', 'wb') as video:
        video.write(response.content)

    # Тута определение скорости
    max_speed = 40

    if max_speed > 30:
        url = f'http://localhost:5000/api/events/{id}/sub_label'
        data = {
            "subLabel": f"Макс. скорость: {max_speed} км/ч"
        }

        response = requests.post(url=url, json=data)
        response.json()

In [6]:
def set_retain_to_true(id):
    url = f'http://localhost:5000/api/events/{id}/retain'
    # set star
    response = requests.post(url=url)
    # remove star
    # response = requests.delete(url=url)
    print(response.json())

def set_sub_label(id, sublabel):
    url = f'http://localhost:5000/api/events/{id}/sub_label'
    data = {
            "subLabel": sublabel
        }
    # Set sublabel
    response = requests.post(url=url, json=data)
    print(response.json())
# set_retain_to_true('1711048455.571456-ywyz75')

In [ ]:
1711125895.477197-tt130p 1711125895.477197 None         no_vest test ['zone_1']
{id} {start_time} {end_time} {label} {camera} {entered_zones}

In [7]:
# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, reason_code, properties):
    print(f"Connected with result code {reason_code}")
    # Subscribing in on_connect()
    client.subscribe("frigate/events")

# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    """
    The callback for when a PUBLISH message is received from the server.
    """
    if (msg.topic == 'frigate/events'):
        payload = json.loads(str(msg.payload)[2:-1])

        # Required alues from payload
        id = payload["after"]["id"]
        start_time = payload["after"]["start_time"]
        end_time = payload["after"]["end_time"]
        label = payload["after"]["label"]
        camera = payload["after"]["camera"]
        entered_zones = payload["after"]["entered_zones"]
        sub_label = payload["after"]["sub_label"]

        print(f'{msg.topic} {id} {start_time} {end_time} \
        {label} {camera} {entered_zones}')

        # Unwanted object from camera in zone
        for entered_zone in entered_zones:
            if label in LABELS and camera in CAMERAS and entered_zone in ZONES:
                sublabel_to_set = 'Unwanted object from camera in zone'
                set_retain_to_true(id)
                set_sub_label(id, sublabel_to_set)
                print('Unwanted object from camera in zone')
                break

        # Event longer than stated
        if end_time is not None:
            if end_time - start_time > DURATION:
                sublabel_to_set = 'Event longer than stated'
                set_retain_to_true(id)
                if sub_label is not None:
                    set_sub_label(id, sublabel_to_set)
                print('Event longer than stated')

        # Speed estimation
        if end_time is not None:
            # speed_estimation(id)
            print('Speed estimation')
    else:
        print(msg.topic+" "+str(msg.payload))

mqttc = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)
mqttc.on_connect = on_connect
mqttc.on_message = on_message

mqttc.connect("localhost", 1883, 60)

# Blocking call that processes network traffic, dispatches callbacks and
# handles reconnecting.
# Other loop*() functions are available that give a threaded interface and a
# manual interface.
mqttc.loop_forever()

Connected with result code Success
frigate/events 1711126617.025673-m8vd1l 1711126617.025673 None         person test []
frigate/events 1711126617.025673-w2c6h8 1711126617.025673 None         helmet test []
frigate/events 1711126617.025673-cwinqa 1711126617.025673 None         person test []
frigate/events 1711126617.025673-w2c6h8 1711126617.025673 1711126623.391656         helmet test []
Speed estimation
frigate/events 1711126617.025673-cwinqa 1711126617.025673 None         person test []
frigate/events 1711126617.025673-m8vd1l 1711126617.025673 None         person test []
frigate/events 1711126627.228192-z7gsz8 1711126627.228192 None         helmet test []
frigate/events 1711126627.228192-z7gsz8 1711126627.228192 1711126628.033905         helmet test []
Speed estimation
frigate/events 1711126617.025673-m8vd1l 1711126617.025673 None         person test []
frigate/events 1711126617.025673-cwinqa 1711126617.025673 None         person test []
frigate/events 1711126633.42482-e2tg0r 171112

KeyboardInterrupt: 

In [ ]:
print(f'{12} {12} {45} {13} \
{6546546} {8767} {454}')

12 12 45 13 6546546 8767 454
